### User-base Collaborative Filtering - Retrieval
This notebook demonstrates how to retrieve the recommendations for a user using the Item-based Collaborative Filtering model in production. 

#### Pre-requisites
1. Have the index trained and saved in the `yelp_UserCF.db` file.

In [1]:
import sqlite3
import pickle

In [2]:
# Connect to the SQLite database
db_path = './yelp_UserCF.db'
conn = sqlite3.connect(db_path)

In [3]:
# Function to retrieve user-business mappings from the database
def retrieve_user_user_mapping(conn):
    cursor = conn.cursor()

    # Fetch user mappings
    cursor.execute('''SELECT user_id, user_idx FROM user_mapping''')
    user_mapping = {row[0]: row[1] for row in cursor.fetchall()}

    return user_mapping

In [4]:
# Retrieve the user-business mappings
user_mapping = retrieve_user_user_mapping(conn)

In [5]:
# Function to get businesses a user interacted with
def get_user_businesses(user_id, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT business_id, stars_review FROM user_item_index WHERE user_id = ?''', (user_id,))
    return cursor.fetchall()

In [6]:
# Function to get top-k similar users for a given user
def get_top_k_similar_users(user_id, k, conn):
    cursor = conn.cursor()
    cursor.execute('''SELECT similarity_vector FROM user_user_similarity WHERE user_id = ?''', (user_id,))
    result = cursor.fetchone()

    if result is None:
        return []

    similarity_vector = pickle.loads(result[0])
    indices, data = similarity_vector

    # Get top-k similar users
    top_k = sorted(zip(indices, data), key=lambda x: -x[1])[:k]

    # Map indices to user ids
    similar_users = [(list(user_mapping.keys())[idx], score) for idx, score in top_k]

    return similar_users

In [7]:
# Function to predict user interests based on similar users
def predict_user_interests_usercf(user_id, k=10, conn=conn):
    # Get top-k similar users
    similar_users = get_top_k_similar_users(user_id, k, conn)

    recommended_businesses = {}
    
    # For each similar user, get their business interactions
    for similar_user_id, _ in similar_users:
        similar_user_businesses = get_user_businesses(similar_user_id, conn)

        for business_id, score in similar_user_businesses:
            if business_id in recommended_businesses:
                recommended_businesses[business_id] += score
            else:
                recommended_businesses[business_id] = score

    # Sort recommendations by score
    recommended_businesses = sorted(recommended_businesses.items(), key=lambda x: -x[1])

    return recommended_businesses[:k]

In [8]:
# Get the top 10 users in the database
# user_ids = list(user_mapping.keys())[:10]

user_ids = ['razUB7ciYZluvxWM6shmtw', '3YhG4h4Ok654iVfqdmkuRg', 'VyC2fG4dcMG07nrxh4jLnw', 'Q5jOFJYhIsN8ouJ1rnsLQQ', 'gdcRlubKDmslUYFPHUp1Cg', 'CNeaJDD_ZihiafOvSGSpPw', 'TItDMAEw7_6Nge38jJPspA', 'zxyKewY6p0CjnFprEutaog', 'IsdzdZEH9uHTnwX3acqhtg', 'AC-oO5luq3enSNFDYAVKJA']

for user_id in user_ids:
    recommendations = predict_user_interests_usercf(user_id, k=10, conn=conn)
    print(f"Recommendations for user {user_id}: {recommendations}")

# Close the database connection when done
conn.close()

Recommendations for user razUB7ciYZluvxWM6shmtw: [('neiMxOk7V4Zgq5zsvuWTaQ', 10.0), ('KIjOP0a4gTe-zRdnVCgsGA', 10.0), ('tYqIKRQjDCywDVkbfUGSug', 10.0), ('wpQojdbIpedF8a1PNokytQ', 10.0), ('qlt0aeVnh4B3X5mUeEUtnQ', 9.0), ('297f-8AdTbu-wwVedvac6w', 9.0), ('jF3RPKNsdcb4657pNRbGxQ', 9.0), ('kkcQYuF3w5iHnHMf0EnRhQ', 9.0), ('rDh6j39BYX4IL9RSZDGs2A', 9.0), ('nwVZnjRDhUrFo1kGahLIZQ', 8.0)]
Recommendations for user 3YhG4h4Ok654iVfqdmkuRg: [('VXPpzhD7mA262gIv1T0WPQ', 19.0), ('f9_TLVlUHBv0869CygEbZg', 17.0), ('Eq5w0ZAW0PV30nNkJxJY_A', 13.0), ('HCUSKJ9NUkISn31O8Jgj_w', 12.0), ('GYgJtmk1v06FJesYYjrDww', 12.0), ('PVDh-6ge94GefpNfXzsSGw', 11.0), ('2fTfpN5SggLgW4LlzptMPg', 10.0), ('mMnkbIMG9MUW-rb5a5Q8Og', 10.0), ('4-HJQtuLSQ0olOxfGjwG5A', 10.0), ('Esf3-D_44pArPd9GqysoCg', 10.0)]
Recommendations for user VyC2fG4dcMG07nrxh4jLnw: [('NnlXgLZ93KPj1wYSiqjyIg', 17.0), ('j4KA-BujEH7xmG6bD5E-gQ', 14.0), ('l4IeGBHXV2E8S8COd9rx9A', 13.0), ('-B6XL-ZWsVHlAQyYcd3eEg', 10.0), ('0IG9w6YCkh-z5pnfSip6Nw', 9.0), ('Ml3En